Customizing Workflows
===================

This notebook shows how to write workflows for BQSKit, using the Hubbard Model$^1$ as an examplar. The Hubbard Model is only briefly introduced; full details on the Hubbard Model, the Variational Quantum Eigensolver, the effects of noise, and more, can be found as part of the scikit-quant tutorials.$^2$

Prerequisites: OpenFermion, Qiskit, BQSKit, and SciPy (which will pull in NumPy), e.g. install with pip:

`$ python -m pip install openfermion qiskit bqskit scipy`

Introduction
--------------

We will solve the Hubbard Model using the Variational Quantum Eigensolver (VQE) algorithm. The Hubbard Model (HM) has a range of uses, most notably in solid-state physics to explain phase transitions and other correlated behaviors. There are several variations of the HM, but in essence, the model describes interacting particles on a lattice. Despite its apparent simplicity, it is hard to solve on a classical computer because the known algorithms scale exponentially unless further approximations are made. In this tutorial, we will use the VQE algorithm to estimate the energy ground states of different configurations of a HM.

Basic Fermi-Hubbard Model
---------------------------------

We will restrict ourselves to a Hubbard Model with fermions (such as electrons), or the Fermi-Hubbard Model. A visualization of an example with a 2x2 grid with spinful electrons is shown below. Electrons being fermions means that two electrons can not occupy the same quantum state. Because the electrons have spin, there can thus be at most two electrons per site (one spin-up, one spin-down). The model is characterized by an interaction term 'U', which only contributes if a spin-up and a spin-down electron occupy the same site; and by a "hopping" term 't', representing the kinetic energy in the system. In this basic model, only hops to neighboring sites are considered.

<table style="padding: 0px"><tr></tr><tr>
<td style="width: 65%; padding: 0px">
<div align="left" style="text-align: left; font-size: 120%">
The energy in the model is described by the following Hamiltonian using fermion algebra:
<br>
<p>\begin{equation*}
H = -t \sum_{ij,\sigma=\uparrow\downarrow} {a_{i\sigma}^\dagger a_{j\sigma}} + U \sum_i {a_{i\uparrow}^\dagger a_{i\uparrow} a_{i\downarrow}^\dagger a_{i\downarrow}}
\end{equation*}</p>
where:
<ul>
    <li>$a_{i\sigma}^\dagger$ = creation operator for site 'i' and spin $\sigma$</li>
    <li>$a_{i\sigma}$ = annihilation operator for site 'i' and spin $\sigma$</li>
    <li>U = interaction term</li>
    <li>t = hopping term</li>
</ul>
<p>The first sum quantifies moving electrons from each site 'j' to each neighboring site 'i' (first term, that is: $a_i^\dagger$ "creates" an electron at site 'i' while $a_j$ simultaneously "annihilates" it from site 'j'), and vice versa (second term), considering both up and down spins ($\sigma$) independently; the second sum quantifies the interaction contribution for having two electrons, one up and one down, at the same site 'i'.
</p></div></td>
    <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
<td style="width: 35%">
    <img src='./images/basic_fermi_hubbard_model.png'>
</td>
</tr></table>

<p>
Although the pictorial shows the electrons as concrete particles, the model treats them as waves that extend over all sites. A solution to the model provides the probabilities with which any of the electrons can be measured, as a particle, at each one of the sites.

Mapping fermions to qubits
---------------------------------

Fermion algebra is only a subset of what a quantum computer can express, since fermions are interchangeable and many states are thus completely equivalent, whereas qubits are identifiable (each one has a well defined location on a chip or ion trap, for example). Thus, it is useful to perform a mapping from fermion to qubit algebra that preserves the fermionic relations, to ensure that the mapped operators only produce states inside that fermionic subset; meaning that the computation should not, noise aside, produce non-physical states. Doing so reduces the scope for errors.$^3$

There are several possible mappings from fermion to qubit operators that preserve the fermion algebra. Here, we'll consider the Jordan-Wigner$^4$ and Bravyi-Kitaev$^5$ transforms, because these are commonly used and easy to understand and their differences$^6$ show an important point about how the optimizability of circuits.

In [1]:
%load_ext autoreload

In [2]:
# useful tools
import logging
import numpy as np
import qiskit as qk
import scipy
from timeit import default_timer as timer

# BQSKit tooling
import bqskit
from bqskit.ext import qiskit_to_bqskit

# VQE objective function for Hamiltonians
from hubbard import EnergyObjective, hamiltonian_qiskit, hubbard

In [3]:
def display_circuit(circuit, basis=['u', 'cx']):
    """perform basic, unoptimized decomposition for circuit display"""

    uc =  qk.transpile(circuit, basis_gates=basis)
    
    for gate, count in uc.count_ops().items():
        print("gate %s count: %d" % (gate, count))

    return uc.draw()

In [4]:
# Setup for a 2x1 grid with a single electron; this is very small model with
# 2 classical optimization parameters. You can try a larger model, but this
# small example is chosen to easily run on a laptop on battery power.
hm_lattice = {
    'x_dimension' : 2,
    'y_dimension' : 1,      # 2D x_dim * y_dim grid
    'tunneling'   : 1.0,    # hpping term `t`
    'coulomb'     : 2.0,    # interaction term `U`
    'spinless'    : False,  # double negative, so do have spins
    'periodic'    : False,  # do not allow wrap-around hopping
}

hm_filling = {
    'n_electrons_up'   : 1,
    'n_electrons_down' : 0,
}

Energy Objective
--------------------

The Variational Quantum Eigensolver is an iterative algorithm where a classical optimizer minimizes an energy objective (the expectation value of the Hamiltonian) as calculated on the quantum computer. Typically, an Hamiltonian has many components, which can be evaluated in parallel, on seperate quantum chips/computers. The final result is obtained by summing the parts classically using their proper weights.
<table style="padding: 0px"><tr></tr><tr>
<td style="width: 50%; padding: 0px">
<div align="left" style="text-align: left; font-size: 120%">
The trial quantum states proposed by the classical optimizer are constructed using an <i>Ansatz</i> that is parametrized using classical values only.
Here, we wil use <i>Unitary Coupled Cluster</i> Ansatz for single and double excitations (UCCSD). Briefly, this Ansatz explores the search space by moving single electrons and by swapping pairs.
This Ansatz is easier on the classical optimizer than many alternatives, but produces rather deep circuits, which is why the use of a tool like BQSKit becomes a necessity.
</div></td>
<td><img src='./images/vqe_structure.png'></td>
</tr></table>
To complete a circuit, we need to fill it with electrons and then instantiate the circuit for a set of optimization paremeters. Note that the meaning of these parameters depends on mapping.

In [5]:
# Hubbard Model Hamiltonian using Jordan-Wigner mapping
hm_jw = hamiltonian_qiskit(fermion_transform='jordan-wigner', **hm_lattice)

# create an energy objective, which can be minimized classically, from the mapped hamiltonian
jw_objective = EnergyObjective(hm_jw, **hm_filling)

# instantiate a circuit from a set of classical parameters
jw_circuit_instance = jw_objective.generate_circuit([0.1]*jw_objective.npar())

# visualize the circuit, using basic gates, and collect some statistics
c_jw = qk.transpile(jw_circuit_instance, basis_gates=["u3", "cx"])

for gate, count in c_jw.count_ops().items():
    print("gate %s count: %d" % (gate, count))
    
c_jw.draw()

gate cx count: 124
gate u3 count: 110


global phase: 2π
     ┌───────────────┐          ┌───┐┌────────────────┐┌───┐┌────────────────┐»
q_0: ┤ U3(π/2,-π,-π) ├──────────┤ X ├┤ U3(0,0,-0.025) ├┤ X ├┤ U3(0,π/2,-π/2) ├»
     └┬─────────────┬┘     ┌───┐└─┬─┘└────────────────┘└─┬─┘└─────┬───┬──────┘»
q_1: ─┤ U3(π/2,0,π) ├──────┤ X ├──■──────────────────────■────────┤ X ├───────»
      ├─────────────┤ ┌───┐└─┬─┘                                  └─┬─┘       »
q_2: ─┤ U3(π/2,0,π) ├─┤ X ├──■──────────────────────────────────────■─────────»
     ┌┴─────────────┴┐└─┬─┘                                                   »
q_3: ┤ U3(π/2,0,π/2) ├──■─────────────────────────────────────────────────────»
     └───────────────┘                                                        »
«                                                     ┌───┐┌────────────────┐»
«q_0: ────────────────────────────────────────────────┤ X ├┤ U3(0,0,-0.025) ├»
«     ┌────────────────┐                         ┌───┐└─┬─┘└────────────────┘»
«q_1: ┤ U3(0,π/2,-π/2) ├─────────────────────────┤ X ├──■────────────────────»
«     └─────┬───┬──────┘┌──────────────────┐┌───┐└─┬─┘                       »
«q_2: ──────┤ X ├───────┤ U3(π/2,π/2,-π/2) ├┤ X ├──■─────────────────────────»
«           └─┬─┘       ├──────────────────┤└─┬─┘                            »
«q_3: ────────■─────────┤ U3(π/2,-π/2,π/2) ├──■──────────────────────────────»
«                       └──────────────────┘                                 »
«     ┌───┐┌────────────────┐                                                  »
«q_0: ┤ X ├┤ U3(0,π/2,-π/2) ├──────────────────────────────────────────────────»
«     └─┬─┘└─────┬───┬──────┘┌──────────────────┐                         ┌───┐»
«q_1: ──■────────┤ X ├───────┤ U3(π/2,π/2,-π/2) ├─────────────────────────┤ X ├»
«                └─┬─┘       └──────┬───┬───────┘┌──────────────────┐┌───┐└─┬─┘»
«q_2: ─────────────■────────────────┤ X ├────────┤ U3(π/2,-π/2,π/2) ├┤ X ├──■──»
«                                   └─┬─┘        └┬────────────────┬┘└─┬─┘     »
«q_3: ────────────────────────────────■───────────┤ U3(0,π/2,-π/2) ├───■───────»
«                                                 └────────────────┘           »
«     ┌───┐┌───────────────┐┌───┐┌────────────────┐                  »
«q_0: ┤ X ├┤ U3(0,0,0.025) ├┤ X ├┤ U3(0,π/2,-π/2) ├──────────────────»
«     └─┬─┘└───────────────┘└─┬─┘└─────┬───┬──────┘┌────────────────┐»
«q_1: ──■─────────────────────■────────┤ X ├───────┤ U3(0,π/2,-π/2) ├»
«                                      └─┬─┘       └─────┬───┬──────┘»
«q_2: ───────────────────────────────────■───────────────┤ X ├───────»
«                                                        └─┬─┘       »
«q_3: ─────────────────────────────────────────────────────■─────────»
«                                                                    »
«                                   ┌───┐┌────────────────┐┌───┐»
«q_0: ──────────────────────────────┤ X ├┤ U3(0,0,-0.025) ├┤ X ├»
«                              ┌───┐└─┬─┘└────────────────┘└─┬─┘»
«q_1: ─────────────────────────┤ X ├──■──────────────────────■──»
«     ┌──────────────────┐┌───┐└─┬─┘                            »
«q_2: ┤ U3(π/2,π/2,-π/2) ├┤ X ├──■──────────────────────────────»
«     ├──────────────────┤└─┬─┘                                 »
«q_3: ┤ U3(π/2,π/2,-π/2) ├──■───────────────────────────────────»
«     └──────────────────┘                                      »
«     ┌────────────────┐                                           ┌───┐»
«q_0: ┤ U3(0,π/2,-π/2) ├───────────────────────────────────────────┤ X ├»
«     └─────┬───┬──────┘┌────────────────┐                    ┌───┐└─┬─┘»
«q_1: ──────┤ X ├───────┤ U3(π/2,π/2,-π) ├────────────────────┤ X ├──■──»
«           └─┬─┘       └─────┬───┬──────┘ ┌────────────────┐ └─┬─┘     »
«q_2: ────────■───────────────┤ X ├────────┤ U3(0,π/2,-π/2) ├───■───────»
«                             └─┬─┘       ┌┴────────────────┴┐          »
«q_3: ──────────────────────────■─────────┤ U3(π/2,-π/2,π/2) ├──────────

**Exercise:** Do the same, but now for the Bravyi-Kitaev mapping. Compare the circuits, find the main differences in structure, and try to guess how that might affect the synthesis.  

In [6]:
# Hubbard Model Hamiltonian using Bravyi-Kitaev mapping
hm_bk = hamiltonian_qiskit(fermion_transform='bravyi-kitaev', **hm_lattice)

# create an energy objective, which can be minimized classically, from the mapped hamiltonian
bk_objective = EnergyObjective(hm_bk, **hm_filling)

# instantiate a circuit from a set of classical parameters
bk_circuit_instance = bk_objective.generate_circuit([0.1]*bk_objective.npar())

# visualize the circuit, using basic gates, and collect some statistics
c_bk = qk.transpile(bk_circuit_instance, basis_gates=["u3", "cx"])

for gate, count in c_bk.count_ops().items():
    print("gate %s count: %d" % (gate, count))
    
c_bk.draw()

gate cx count: 90
gate u3 count: 80


global phase: 4.6124
      ┌───────────────┐      ┌───┐┌──────────────┐┌───┐┌────────────────┐»
q_0: ─┤ U3(π/2,-π,-π) ├──────┤ X ├┤ U3(0,0,0.05) ├┤ X ├┤ U3(0,π/2,-π/2) ├»
     ┌┴───────────────┴┐┌───┐└─┬─┘└──────────────┘└─┬─┘└─────┬───┬──────┘»
q_1: ┤ U3(π/2,-π,-π/2) ├┤ X ├──■────────────────────■────────┤ X ├───────»
     └─┬─────────────┬─┘└─┬─┘                                └─┬─┘       »
q_2: ──┤ U3(π/2,0,π) ├────■────────────────────────────────────■─────────»
       └┬───────────┬┘                                                   »
q_3: ───┤ U3(π,0,π) ├────────────────────────────────────────────────────»
        └───────────┘                                                    »
«                              ┌───┐┌────────────────┐┌───┐┌────────────────┐»
«q_0: ─────────────────────────┤ X ├┤ U3(0,0,-0.025) ├┤ X ├┤ U3(0,π/2,-π/2) ├»
«      ┌────────────────┐ ┌───┐└─┬─┘└────────────────┘└─┬─┘└─────┬───┬──────┘»
«q_1: ─┤ U3(π/2,π/2,-π) ├─┤ X ├──■──────────────────────■────────┤ X ├───────»
«     ┌┴────────────────┴┐└─┬─┘                                  └─┬─┘       »
«q_2: ┤ U3(π/2,π/2,-π/2) ├──■──────────────────────────────────────■─────────»
«     └──────────────────┘                                                   »
«q_3: ───────────────────────────────────────────────────────────────────────»
«                                                                            »
«                                 ┌───┐┌────────────────┐┌───┐»
«q_0: ────────────────────────────┤ X ├┤ U3(0,0,-0.025) ├┤ X ├»
«                            ┌───┐└─┬─┘└────────────────┘└─┬─┘»
«q_1: ───────────────────────┤ X ├──■──────────────────────■──»
«     ┌────────────────┐┌───┐└─┬─┘                            »
«q_2: ┤ U3(0,π/2,-π/2) ├┤ X ├──■──────────────────────────────»
«     └────────────────┘└─┬─┘                                 »
«q_3: ────────────────────■───────────────────────────────────»
«                                                             »
«     ┌────────────────┐                                   ┌───┐»
«q_0: ┤ U3(0,π/2,-π/2) ├───────────────────────────────────┤ X ├»
«     └─────┬───┬──────┘┌───────────────┐                  └─┬─┘»
«q_1: ──────┤ X ├───────┤ U3(π/2,0,π/2) ├────────────────────┼──»
«           └─┬─┘       └─────┬───┬─────┘┌────────────────┐  │  »
«q_2: ────────■───────────────┤ X ├──────┤ U3(0,π/2,-π/2) ├──■──»
«                             └─┬─┘      └────────────────┘     »
«q_3: ──────────────────────────■───────────────────────────────»
«                                                               »
«     ┌────────────────┐┌───┐┌────────────────┐     ┌───┐┌────────────────┐»
«q_0: ┤ U3(0,0,-0.025) ├┤ X ├┤ U3(0,π/2,-π/2) ├─────┤ X ├┤ U3(0,0,-0.025) ├»
«     └────────────────┘└─┬─┘└────────────────┘     └─┬─┘└────────────────┘»
«q_1: ────────────────────┼───────────────────────────┼────────────────────»
«                         │  ┌────────────────┐┌───┐  │                    »
«q_2: ────────────────────■──┤ U3(0,π/2,-π/2) ├┤ X ├──■────────────────────»
«                            └────────────────┘└─┬─┘                       »
«q_3: ───────────────────────────────────────────■─────────────────────────»
«                                                                          »
«     ┌───┐┌──────────────────┐                       ┌───┐┌──────────────┐»
«q_0: ┤ X ├┤ U3(π/2,π/2,-π/2) ├───────────────────────┤ X ├┤ U3(0,0,0.05) ├»
«     └─┬─┘└──────────────────┘                  ┌───┐└─┬─┘└──────────────┘»
«q_1: ──┼────────────────────────────────────────┤ X ├──■──────────────────»
«       │         ┌───┐        ┌────────────────┐└─┬─┘                     »
«q_2: ──■─────────┤ X ├────────┤ U3(0,π/2,-π/2) ├──■───────────────────────»
«                 └─┬─┘        └────────────────┘                          »
«q_3: ──────────────■──────────────────────────────────────────────────────»
«                                                                          »
«     ┌───┐┌────────────────┐        

Effect of Fermion to Qubit Mapping
-----------------------------------------

The mapping has to encode the positions of the electrons and do so with a guarantee of anti-symmetry of the fermionic wave functions, to respect fermion algebra. There are thus two pieces of information: the electron occupancy on each site and the overall parity. The Jordan-Wigner mapping fully localizes the occupation number (each qubit represents the occupation on a site), which then causes the parity to be completely non-localized: whenever an electron "hops" sites, the parity update requires an update to all qubits, propagated through a chain of controlled gates. The Bravyi-Kitaev transform encodes both occupancy and parity information non-locally, leading to a more balanced result. Check the circuits and find that the Jordan-Wigner mapping results in many series of CNOTs involving all qubits than does Bravyi-Kitaev.

<div class="alert alert-block alert-info">
<b>Note:</b> the circuits above where generated using the same number of Trotter steps (a method of approximating an exponential) and although the Bravyi-Kitaev circuit is significantly shorter in that case, the approximation is actually slightly worse. After adjusting the number of steps and the method to achieve the same quality of approximation (use `trotter_steps=2, trotter_mode='trotter'` for Jordan-Wigner and `trotter_steps=3, trotter_mode='trotter'`), both methods are much closer in the number of CNOTs in the unoptimized circuit. Since Trotter steps are repetitions, this does not change any conclusions about the relative effectiveness of BQSKit's circuit optimizer.

**Exercise:** Use what you learned in the __[BQSKit introductory notebook](1_comping_with_bqskit.ipynb)__ to compile both circuit instances. Try for `optimization_level=2` and `max_synthesis_size=3` first, adjust and play with the parameters if time allows (do note that larger block size and/or higher optimization level can result in significantly longer compilation times). Compare and explain the results.

**Optional exercise:** The examples so far have presumed all-to-all. Experiment with one of the realistic QPU models, or your own. Are the results and conclusions the same?

In [7]:
circuits = ((c_jw, 'Jordan-Wigner'), (c_bk, 'Bravyi-Kitaev'))

for circuit, mapping in circuits:
    opt_circuit = bqskit.compile(qiskit_to_bqskit(circuit), optimization_level=2, max_synthesis_size=3)

    # Print new circuit statistics
    print("Compiled Circuit Statistics for:", mapping)
    print("  Gate Counts:", opt_circuit.gate_counts)
    print("  Connectivity:", opt_circuit.coupling_graph)

Compiled Circuit Statistics for: Jordan-Wigner
  Gate Counts: {CNOTGate: 108, U3Gate: 121}
  Connectivity: CouplingGraph({(2, 3), (1, 2), (0, 1)})
Compiled Circuit Statistics for: Bravyi-Kitaev
  Gate Counts: {CNOTGate: 66, U3Gate: 79}
  Connectivity: CouplingGraph({(0, 1), (1, 2), (2, 3), (0, 2), (1, 3)})


Variational Quantum Eigensolver
--------------------------------------

In the VQE algorithm, circuits are synthesized based on parameters provided by a classical minimizer. All we need to do, is to pass the energy objective function to a classical optimizer. Since we are not considering noise here (see the [scikit-quant tutorials](https://github.com/scikit-quant/scikit-quant/tree/master/tutorials), if interested), most any optimizer will do. Here, we will use [BFGS from SciPy](https://docs.scipy.org/doc/scipy/reference/optimize.html#module-scipy.optimize).

In [8]:
min_energy = scipy.optimize.minimize(jw_objective, [0.1]*jw_objective.npar(), method='BFGS')
print("minimum energy: %.5f (exact: %.5f)" % (min_energy.fun, hubbard.exact(hm_jw, **hm_filling)))

minimum energy: -1.00000 (exact: -1.00000)


**Exercise:** Verify the results for the objective using the Bravyi-Kitaev mapping. Optionally, try different fillings and grid sizes (note that larger grids will have local minima where the optimizer can get stuck if not seeded close to the global minimum).

In [9]:
min_energy = scipy.optimize.minimize(bk_objective, [0.1]*bk_objective.npar(), method='BFGS')
print("minimum energy: %.5f (exact: %.5f)" % (min_energy.fun, hubbard.exact(hm_bk, **hm_filling)))

minimum energy: -1.00000 (exact: -1.00000)


The Ansatz forms the bulk of any VQE circuit: both the input state and the Hamiltonian components only require single qubit rotations, at most, at the beginning and at the very end to measure the desired projection components for each qubit.
It thus makes sense to optimize the Ansatz part of the circuit independently and re-use it (with at most a fast peep-hole pass to merge single qubit gates at start and end where possible).

**Exercise:** The `EnergyObjective` is written such that a derived class can implement the `bqskit` optimization. Fill in the remainder below, and verify the results. Use the added pass to collect statistics on the `bqskit` results.

In [10]:
from hubbard import EnergyObjective
from bqskit.compiler.basepass import BasePass

# to show BQSKit is running
log = logging.getLogger("bqskit.compiler")
log.setLevel(logging.INFO)

class MyObjective(EnergyObjective):
    def optimize_evolution(self, bqskit_circuit):
        return bqskit.compile(bqskit_circuit, optimization_level=2, max_synthesis_size=3)
    
bk_objective = MyObjective(hm_bk, **hm_filling)
start = timer()
min_energy = scipy.optimize.minimize(bk_objective, [0.1]*bk_objective.npar(), method='BFGS')
print("converged in %.0f seconds" % (timer() - start))
print("minimum energy: %.5f (exact: %.5f)" % (min_energy.fun, hubbard.exact(hm_bk, **hm_filling)))

(8674057728): 23:20:40.810 - INFO     | bqskit.compiler.compiler: Started compiler process.


INFO:bqskit.compiler.compiler:Started compiler process.


(8674057728): 23:20:40.811 - INFO     | bqskit.compiler.compiler: Compiling task: f36e075c-d374-4897-81db-398871088014


INFO:bqskit.compiler.compiler:Compiling task: f36e075c-d374-4897-81db-398871088014


(8674057728): 23:20:40.821 - INFO     | bqskit.compiler.compiler: Submitted task: f36e075c-d374-4897-81db-398871088014


...

(8674057728): 23:43:13.908 - INFO     | bqskit.compiler.compiler: Submitted task: 3e9d1b0c-cb9b-4d42-8506-8dc7b95e050a


INFO:bqskit.compiler.compiler:Submitted task: 3e9d1b0c-cb9b-4d42-8506-8dc7b95e050a


(8674057728): 23:43:18.935 - INFO     | bqskit.compiler.compiler: Stopped compiler process.


INFO:bqskit.compiler.compiler:Stopped compiler process.


converged in 637 seconds
minimum energy: -0.96911 (exact: -1.00000)


Seeded Synthesis
---------------------

An circuit optimized by BQSKit is no longer parametrized (optimizing a fully instantiate circuit leads to better results than when the parametrization is left in-place in the form of gates). However, many classical optimizers use a form of gradient descent, meaning that the current circuit instance requested is likely to be close to the previos one. Furthermore, even when the optimizer uses a different approach, such as a stencil or grid, where the parameters are further apart, the overall structure of the final optimized current circuit is likely to be close to the previous.

BQSKit can make use of this observation with so-called "seeded" synthesis: use the previous instantiation as a template to synthesize the next circuit.

**Exercise:** Store the result circuit of the current iteration in a data member of the objective. Then next round, use its `instantiate()` method to generate a new circuit. Verify whether the new circuit instantiated from the old template is close enough using a threshold of your choosing. Verify the result from minimization.

In [11]:
from timeit import default_timer as timer

class MyObjective(EnergyObjective):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.template = None

    def optimize_evolution(self, bqskit_circuit):
        opt_circuit = None

        if self.template is not None:
            target_unitary = bqskit_circuit.get_unitary()
            opt_circuit = self.template.instantiate(target_unitary)    
            if 1e-8 < self.template.get_unitary().get_distance_from(target_unitary):
                opt_circuit = None    # reject this circuit
        
        if opt_circuit is None:
            opt_circuit = bqskit.compile(bqskit_circuit, optimization_level=2, max_synthesis_size=3)
        
        self.template = opt_circuit

        return opt_circuit
 
bk_objective = MyObjective(hm_bk, **hm_filling)
start = timer()
min_energy = scipy.optimize.minimize(bk_objective, [0.1]*bk_objective.npar(), method='BFGS')
print("converged in %.0f seconds" % (timer() - start))
print("minimum energy: %.5f (exact: %.5f)" % (min_energy.fun, hubbard.exact(hm_bk, **hm_filling)))

(8674057728): 23:43:22.092 - INFO     | bqskit.compiler.compiler: Started compiler process.


INFO:bqskit.compiler.compiler:Started compiler process.


(8674057728): 23:43:22.093 - INFO     | bqskit.compiler.compiler: Compiling task: b1f36035-4fce-4575-bdc3-58bf3903183d


INFO:bqskit.compiler.compiler:Compiling task: b1f36035-4fce-4575-bdc3-58bf3903183d


(8674057728): 23:43:22.102 - INFO     | bqskit.compiler.compiler: Submitted task: b1f36035-4fce-4575-bdc3-58bf3903183d


...

INFO:bqskit.compiler.compiler:Started compiler process.


(8674057728): 10:04:59.147 - INFO     | bqskit.compiler.compiler: Compiling task: 8742a3e1-27eb-4627-8966-cf6c792e2ede


INFO:bqskit.compiler.compiler:Compiling task: 8742a3e1-27eb-4627-8966-cf6c792e2ede


Failed in optimize_evolution: No valid workers found


TimeoutError: No valid workers found

Optional Excercise
----------------------

The use of BQSKit truly comes into its own when there is noise present in the system, as is the case in on any current, real, QPU: the circuit optimization will show real improvements in the final result. As a note of caution, however: when noise is present, several things change. For example, a noise-resilient optimizer is needed and even then, the final energy result obtained is likely to be higher than the global minimum due to the effect of noise, even when the final parameters are spot-on. Additionally, simulating this will take considerably longer, and multiple runs are needed for convincing statistics.

Install scikit-quant, to make it available as a Qiskit optimizer:

    `$ python -m pip install scikit-quant`

Construct a simple noise model (the circuits will, even after optimization, be too long to be run on current hardware; you can try it by setting `noise_model='realistic'`) and pass it through the constructor of the `EnergyObjective`. Run with the base `EnergyObjective` and with your derived implementation using BQSKit. Compare the results with and without `bqskit` optimization.

In [ ]:
from qiskit.algorithms.optimizers import IMFIL
from hubbard import noise_model

logging.getLogger('hubbard').setLevel(logging.INFO)

In [ ]:
noise = noise_model.create(p_gate_1q=0.01, p_gate_2q=0.02)

In [ ]:
objective = EnergyObjective(hm_bk, noise_model=noise, **hm_filling)
npar = objective.npar()

result = IMFIL(maxiter=100).minimize(objective, np.array([0.1]*npar), bounds=np.array([(-1,1)]*npar))
print("minimum energy: %.5f (at solution %s: %.5f; exact: %.5f)" % (
    result.fun, result.x, EnergyObjective(hm_bk, **hm_filling)(result.x), hubbard.exact(hm_bk, **hm_filling)))

In [ ]:
objective = MyObjective(hm_bk, noise_model=noise, **hm_filling)
npar = objective.npar()

result = IMFIL(maxiter=100).minimize(objective, np.array([0.1]*npar), bounds=np.array([(-1,1)]*npar))
print("minimum energy: %.5f (at solution %s: %.5f; exact: %.5f)" % (
    result.fun, result.x, EnergyObjective(hm_bk, **hm_filling)(result.x), hubbard.exact(hm_bk, **hm_filling)))

## References
-------------

[1] J. Hubbard, Proc. Roy. Soc. (Lond.) 276 (1963) 238
<br>[2] https://github.com/scikit-quant/scikit-quant/tree/master/tutorials
<br>[3] https://arxiv.org/abs/cond-mat/0012334, Ortiz et al., "Quantum Algorithms for Fermionic Simulations"
<br>[4] P. Jordan and E. Wigner, "Über das Paulische Äquivalenzverbot", Z. Phys. 47, 631 (1928)
<br>[5] https://arxiv.org/abs/quant-ph/0003137 S. Bravyi, A. Kitaev, "Fermionic quantum computation"
<br>[6] https://arxiv.org/abs/1812.02233, A. Tranter, et al., "A comparison of the Bravyi-Kitaev and Jordan-Wigner transformations for the quantum simulation of quantum chemistry"